# Building a content-based recommender using tf-idf

#### Import libraries and data

In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer #obtaining tdf vectors
from itertools import combinations #finding combination of genres for a given movie
from sklearn.metrics.pairwise import cosine_similarity #To compute the cosine similarities between all tf-idf vectors


df = pd.read_csv("../../Data/ml-latest-small/PreprocessedData_ml_latest_year_small.csv",index_col=0)


#### drop duplicates and keep only movieId, title and genres

In [15]:
df['pasteIDandMovie'] = df['title']+str(df['movieId'])
df = df.drop_duplicates(subset=['pasteIDandMovie'])

movies = df[['movieId', 'title', 'genres']].sort_values(by=['movieId']).reset_index(drop=True)


In [16]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


#### Transform the genres into a meaningful representation of numbers using TfidVectorizer

In [17]:
tf = TfidfVectorizer(stop_words = 'english', analyzer = 'word')
tfidf_matrix = tf.fit_transform(movies['genres'])

In [18]:
##to get an impression of what the result looks like, uncomment the line below
#pd.DataFrame(tfidf_matrix.todense(), columns=tf.get_feature_names(), index=movies.title).sample(10, axis=1).sample(10, axis=0)


#### Use cosine similarity to find similar vectors

In [19]:
cosine_sim = cosine_similarity(tfidf_matrix)
cosine_sim_df = pd.DataFrame(cosine_sim, index=movies['title'], columns=movies['title'])
print('Shape:', cosine_sim_df.shape)
#find a sample of the result below
cosine_sim_df.sample(5, axis=1).round(2) 

Shape: (9719, 9719)


title,Say Anything... (1989),Catch .44 (2011),"Good, the Bad and the Ugly, The (Buono, il brutto, il cattivo, Il) (1966)",Evil Aliens (2005),Tangerine (2015)
title,,,,,
Toy Story (1995),0.13,0.00,0.20,0.09,0.20
Jumanji (1995),0.00,0.00,0.24,0.00,0.00
Grumpier Old Men (1995),0.88,0.00,0.00,0.18,0.42
Waiting to Exhale (1995),1.00,0.20,0.00,0.16,0.69
Father of the Bride Part II (1995),0.50,0.00,0.00,0.32,0.73
...,...,...,...,...,...
Black Butler: Book of the Atlantic (2017),0.16,0.28,0.18,0.10,0.23
No Game No Life: Zero (2017),0.18,0.00,0.00,0.11,0.26
Flint (2017),0.47,0.43,0.00,0.00,0.68


### find the highest tf-idf score for a given movie, function declaration

In [45]:
def genre_recommendation(query_title):
    """
    Recommends movies based on a similarity dataframe
    Parameters
    ----------
    query_title : Movie title (string)

    """
    items= movies[['title', 'genres']]
    #select column with the input movie title, and change it to numpy array 
    #resulting array of indices indicates the positions of the elements that would be in the first i positions
    sel = cosine_sim_df.loc[:,query_title].to_numpy().argpartition(range(-1,-10,-1)) 
    #resulting subset of column names is ordered in descending order of the corresponding values in the title column. 
    #This subset is then assigned to the variable ct    
    ct = cosine_sim_df.columns[sel[-1:-(10+2):-1]]
    #drop columns title from input and merge the df with the original dataframe. show only first i results. 
    ct = ct.drop(query_title, errors='ignore')
    
    xx = pd.DataFrame(ct).merge(items).head(10)
    
    #add similarity score to xx
    xx['Similarity Score'] = cosine_sim_df.loc[query_title, xx['title']].values
    
    return xx

#### find your movie on what the recommendation should be based on, and paste the name in the function below

In [48]:
# for example, find a harry potter movie
movies[movies["title"].str.contains('Harry')]

,movieId,title,genres
1004,1307,When Harry Met Sally... (1989),Comedy|Romance
1280,1701,Deconstructing Harry (1997),Comedy|Drama
1637,2184,"Trouble with Harry, The (1955)",Comedy|Mystery
2524,3387,Who's Harry Crumb? (1989),Comedy|Mystery
2525,3388,Harry and the Hendersons (1987),Children|Comedy
2526,3389,Let's Get Harry (1986),Action|Adventure
3039,4077,"With a Friend Like Harry... (Harry, un ami qui...",Drama|Thriller
3543,4855,Dirty Harry (1971),Action|Crime|Thriller
3569,4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,Adventure|Children|Fantasy
4070,5816,Harry Potter and the Chamber of Secrets (2002),Adventure|Fantasy


In [49]:
similar_movies=genre_recommendation('Harry Potter and the Order of the Phoenix (2007)')


In [50]:
similar_movies

,title,genres,Similarity Score
0,"Hobbit: An Unexpected Journey, The (2012)",Adventure|Fantasy|IMAX,0.967592
1,Alice in Wonderland (2010),Adventure|Fantasy|IMAX,0.967592
2,Jack the Giant Slayer (2013),Adventure|Fantasy|IMAX,0.967592
3,"Hobbit: The Desolation of Smaug, The (2013)",Adventure|Fantasy|IMAX,0.967592
4,Harry Potter and the Prisoner of Azkaban (2004),Adventure|Fantasy|IMAX,0.967592
5,Thor (2011),Action|Adventure|Drama|Fantasy|IMAX,0.936705
6,"Twilight Saga: Breaking Dawn - Part 2, The (2012)",Adventure|Drama|Fantasy|Romance|IMAX,0.930640
7,Harry Potter and the Goblet of Fire (2005),Adventure|Fantasy|Thriller|IMAX,0.904000
8,Oz the Great and Powerful (2013),Action|Adventure|Fantasy|IMAX,0.902588
9,Thor: The Dark World (2013),Action|Adventure|Fantasy|IMAX,0.902588
